## Image Classification

### 1.Firstly we will do the data preparation

In [1]:
#pip install opencv-python

In [2]:
import os,sys
import cv2

### cv2.imread()
cv2.imread() method loads an image from the specified file. If the image cannot be read (because of missing file, improper permissions, unsupported or invalid format) then this method returns an empty matrix.

Syntax: cv2.imread(path, flag)

#### Parameters:
path: A string representing the path of the image to be read.
flag: It specifies the way in which image should be read. It’s default value is cv2.IMREAD_COLOR

### cv2.imwrite()

OpenCV-Python is a library of Python bindings designed to solve computer vision problems

cv2.imwrite() method is used to save an image to any storage device. 
This will save the image according to the specified format in current working directory.

#### Syntax: cv2.imwrite(filename, image)

#### Parameters:
filename: A string representing the file name. The filename must include image format like .jpg, .png, etc.
image: It is the image that is to be saved.

In [3]:
# Save all the images in the .jpg format, at the same time delete the original images,
def listfiles(rootDir):
  
      list_dirs = os.walk(rootDir)
      for root, dirs, files in list_dirs:
        #print(root,dirs,files)
        #for d in dirs:
            #print(os.path.join(root,d))
      
        for f in files:
            field = f.split('.')[0]
            
            #os.path.join() method in Python join one or more path components intelligently. 
            filepath = os.path.join(root,f)
            try:
                src = cv2.imread(filepath,1)
                print("src=",filepath,src.shape)
                os.remove(filepath)
                cv2.imwrite(os.path.join(root,field + ".jpg"),src)
            except:
                os.remove(filepath)
                continue

In [4]:
rootDir = 'C:\\Users\\zjiax\\Desktop\\Projects\\Image_Classification\\Beak'
listfiles(rootDir)

src= C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak\Google_0000.jpg (640, 640, 3)
src= C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak\Google_0002.jpg (852, 960, 3)
src= C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak\Google_0004.jpg (595, 583, 3)
src= C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak\Google_0005.jpg (300, 300, 3)
src= C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak\Google_0006.jpg (1144, 750, 3)
src= C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak\Google_0009.jpg (602, 640, 3)
src= C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak\Google_0010.jpg (902, 640, 3)
src= C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak\Google_0013.jpg (523, 500, 3)
src= C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak\Google_0014.jpg (392, 519, 3)
src= C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak\Google_0021.jpg (400, 400, 3)
src= C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak\Google

#### Model filtering non-face data
#### Filtering beak-expression pictures artificially

In [5]:
# We should use openCV to detect face
cascade_path = 'haarcascade_frontalface_default.xml'
cascade = cv2.CascadeClassifier(cascade_path)

In [6]:
images  = os.listdir(rootDir)

In [7]:
for image in images:
    im = cv2.imread(os.path.join(rootDir,image),1)
    rects = cascade.detectMultiScale(im,1.3,5)
    print("detect face",len(rects))
    
    if(len(rects) == 0):
       # cv2.namedWindow('Result',0)
       # cv2.imshow("Result",im)
        os.remove(os.path.join(rootDir,image))
        #k = cv2.waitKey(0)
        #if(k==ord('q')):
        # break;

detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 2
detect face 1
detect face 1
detect face 1
detect face 1
detect face 2
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 2
detect face 1
detect face 2
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1
detect face 1


### Hash-based deduplication algorithm for similar graph deduplication

sudo pip install python-Levenshtein

python remove_repeat.py "C:\Users\zjiax\Desktop\Projects\Image_Classification\Beak"

### Data preprocessing

Use OpenCV + Dlib to locate lips' keypoints

In [8]:
import dlib

In [9]:
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"  # Keypoints Detection Model
predictor = dlib.shape_predictor(PREDICTOR_PATH) #Dlib Keypoints Detector
cascade_path = "haarcascade_frontalface_default.xml" ## Face Classifier
cascade = cv2.CascadeClassifier(cascade_path)

In [10]:
def get_landmarks(im):
    rects = cascade.detectMultiScale(im,1,3.5)
    x,y,w,h = rects[0]
    rect = dlib.rectangle(x,y,x+w,y+h)
    return np.matrix([[p.x,p.y] for p in predictor(im,rect).parts()])

In [11]:
## Extract lip area
def getlipfromimage(im,landmarks):
    xmin = 10000
    xmax  = 0
    ymin = 10000
    ymax = 0
    
    for i in range(48,67):
        x = landmarks[i,0]
        y = landmarks[i,1]
        
        if x < xmin: xmin = x
        if x > xmin: xmax = x
        if y < ymin: ymin = y
        if y > ymax: ymax = y
            
    roiwidth  = xmax - xmin
    roiheight = ymax - ymin
    roi = im[ymin:ymax,xmin:xmax,0:3]
    
    # enlarge lip area
    if roiwidth > roiheight:
        dstlen = 1.5 * roiwidth
    else:
        dstlen = 1.5 * roiheight
        
    diff_xlen = dstlen - roiwidth
    diff_ylen = dstlen - roiheight
    newx = xmin
    newy = ymin
    
    imagerows,imagecols,channel = im.shape
    if newx >= diff_xlen/2 and newx + roiwidth + diff_xlen/2 < imagecols:
        newx = newx - diff_len/2
    elif newx < diff_xlen/2:
        newx = 0;
    else:
        newx = imagecols - dstlen;
        
    if newy >=diff_ylen/2 and newy + roiheight + diff_ylen/2 < imagerows:
        newy = newy - diff_ylen/2;
    elif newy < diff_ylen/2:
        newy = 0;
    else:
        newy = imagecols - dstlen;
        
    roi = im[int(newy):int(newy+dstlen),int(newx):int(newx+dstlen),0:3]